In [4]:
!pip install numpy


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [5]:
import numpy as np

In [6]:
dataset_path = './home'
partitioned_data_path = './partitioned_data'

In [7]:
training_set_prop = 0.8
test_set_prop = 0.1
validation_set_prop = 0.1

In [8]:
def data_partition(dataset_path, partitioned_data_path, training_set_prop, test_set_prop, validation_set_prop):
    np.random.randint()